In [1]:
class Tree:
    """A tree with label as its label value."""
    def __init__(self, label, branches=[]):
        self.label = label
        for branch in branches:
            assert isinstance(branch, Tree)
        self.branches = list(branches)

    def __repr__(self):
        if self.branches:
            branch_str = ', ' + repr(self.branches)
        else:
            branch_str = ''
        return 'Tree({0}{1})'.format(self.label, branch_str)

    def __str__(self):
        return '\n'.join(self.indented())

    def indented(self, k=0):
        indented = []
        for b in self.branches:
            for line in b.indented(k + 1):
                indented.append('  ' + line)
        return [str(self.label)] + indented

    def is_leaf(self):
        return not self.branches

In [2]:
class BTree(Tree):
    """A tree with exactly two branches, which may be empty."""
    empty = Tree(None)

    def __init__(self, label, left=empty, right=empty):
        for b in (left, right):
            assert isinstance(b, BTree) or b is BTree.empty
        Tree.__init__(self, label, (left, right))
        
    @property
    def left(self):
        return self.branches[0]

    @left.setter    
    def left(self, value):
        self.branches[0] = value
    
    @property
    def right(self):
        return self.branches[1]

    @right.setter
    def right(self, value):
        self.branches[1] = value
    
    def is_leaf(self):
        return [self.left, self.right] == [BTree.empty] * 2

    def __repr__(self):
        if self.is_leaf():
            return 'BTree({0})'.format(self.label)
        elif self.right is BTree.empty:
            left = repr(self.left)
            return 'BTree({0}, {1})'.format(self.label, left)
        else:
            left, right = repr(self.left), repr(self.right)
            if self.left is BTree.empty:
                left = 'BTree.empty' 
            template = 'BTree({0}, {1}, {2})'
            return template.format(self.label, left, right)

In [3]:
class BSTree:

    @staticmethod
    def construct(labels):
        if not labels:
            return BTree.empty
        tree = BTree(labels[0])
        for label in labels[1:]:
            BSTree.insert(tree, label)
        return tree
    
    @staticmethod
    def insert(tree, label):
        while True:
            if label > tree.label:
                if tree.right is BTree.empty:
                    tree.right = BTree(label)
                    break
                else:
                    tree = tree.right
            else:
                if tree.left is BTree.empty:
                    tree.left = BTree(label)
                    break                    
                else:
                    tree = tree.left
    
    @staticmethod
    def search(tree, key):
        while tree is not BTree.empty:
            if tree.label == key:
                return True
            elif key > tree.label:
                tree = tree.right
            else:
                tree = tree.left
        return False
    
    @staticmethod
    def largest(tree):
        if tree is BTree.empty:
            return None
        while tree.right is not BTree.empty:
            tree = tree.right
        return tree.label
    
    @staticmethod
    def smallest(tree):
        if tree is BTree.empty:
            return None
        if tree.left is BTree.empty:
            return tree.label
        return BSTree.smallest(tree.left)
    
    @staticmethod
    def second(tree):
        if tree.is_leaf():
            return None
        elif tree.right is BTree.empty:
            return BSTree.largest(tree.left)
        elif tree.right.is_leaf():
            return tree.label
        else:
            return BSTree.second(tree.right)
    
    @staticmethod
    def inorder(tree):
        if tree is BTree.empty:
            return []
        else:
            traversal = []
            traversal.extend(BSTree.inorder(tree.left))
            traversal.append(tree.label)
            traversal.extend(BSTree.inorder(tree.right))
            return traversal

In [ ]:
import unittest

class BSTTest(unittest.TestCase):
    
    def setUp(self):
        self.tree = BSTree.construct([18, 15, 17, 21, 19, 13, 24, 20])

    def test_construct(self):
        self.assertEqual(BSTree.inorder(self.tree), [13, 15, 17, 18, 19, 20, 21, 24])
    
    def test_largest(self):
        self.assertEqual(BSTree.largest(self.tree), 24)
        
    def test_smallest(self):
        self.assertEqual(BSTree.smallest(self.tree), 13)
    
    def test_insertion_1(self):
        BSTree.insert(self.tree, 30)
        self.assertEqual(BSTree.inorder(self.tree), [13, 15, 17, 18, 19, 20, 21, 24, 30])
    
    def test_insertion_2(self):
        BSTree.insert(self.tree, 16)
        self.assertEqual(BSTree.inorder(self.tree), [13, 15, 16, 17, 18, 19, 20, 21, 24])
    
    def test_second_1(self):
        self.assertEqual(BSTree.second(self.tree), 21)
    
    def test_second_2(self):
        BSTree.insert(self.tree, 22)
        BSTree.insert(self.tree, 23)
        self.assertEqual(BSTree.second(self.tree), 23)
    
    def test_search_key_that_exists(self):
        self.assertTrue(BSTree.search(self.tree, 19))
        
    def test_search_key_that_not_exists(self):
        self.assertFalse(BSTree.search(self.tree, 14))
        
    
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(BSTTest)
    unittest.TextTestRunner(verbosity=2).run(suite)